## MNIST - Recurrent Neural Network

Inspired from :
    
https://github.com/aymericdamien/TensorFlow-Examples

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('/tmp/data',one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# No. of training cycles and bacth-size
training_epochs = 100000
batch_size = 128
display_step = 50

#Net parameters
n_input = 28
n_steps = 28
n_hidden = 256
n_classes = 10

#Placeholder for feeding the net
x = tf.placeholder("float",[None,n_steps,n_input])
y = tf.placeholder("float",[None,n_classes])

#Weights and biases 
weights = {'out':tf.Variable(tf.random_normal([n_hidden,n_classes]))}
biases = {'out':tf.Variable(tf.random_normal([n_classes]))}


Each image is 28 X 28 pixels. We consider each row at a sequence with 28 time-steps. Hence, each image will have 28 sequences with 28 steps.

#### Understanding the input
I am using a batch size of 128. Hence, the input will have dimensions of (128,28,28). The LSTM cell requires a list of tensors. In our case the list will have 28 tensors(equal to number of time-steps) with each tensor of dimension (128 X 28). The function below reshapes the input, and return the model for RNN.

In [4]:
def RNN1(x,weights,biases):
    x = tf.transpose(x,[1,0,2])
    x = tf.reshape(x,[-1,n_input])
    x = tf.split(0,n_steps,x)
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,forget_bias=1.0)
    
    outputs, states = tf.nn.rnn(lstm_cell,x,dtype=tf.float32,scope='reuse')
    
    return(tf.matmul(outputs[-1],weights['out'])+biases['out'])

In [5]:
# Creating the RNN
pred = RNN1(x,weights,biases)

#Optimization routine
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

#Estimating Accuracy
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

### Running the net

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_epochs:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        batch_x = batch_x.reshape((batch_size,n_steps,n_input))
        
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
        
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict = {x:batch_x,y:batch_y})
            loss = sess.run(cost,feed_dict = {x:batch_x,y:batch_y})
            print ("Iterations :  "+str(step*batch_size) + 
                  ", Minibatch = {:.6f}".format(loss)+
                  ", Training Accuracy = {:.5f}".format(acc))
            
        step += 1
        
    print ("Optimization Finished!")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps,n_input))
    test_label = mnist.test.labels[:test_len]
    
    print ("Testing Accuracy = {}".format(sess.run(accuracy,feed_dict={x:test_data,y:test_label})))
        

Iterations :  6400, Minibatch = 0.565575, Training Accuracy = 0.81250
Iterations :  12800, Minibatch = 0.522545, Training Accuracy = 0.81250
Iterations :  19200, Minibatch = 0.165956, Training Accuracy = 0.93750
Iterations :  25600, Minibatch = 0.308126, Training Accuracy = 0.92188
Iterations :  32000, Minibatch = 0.140764, Training Accuracy = 0.93750
Iterations :  38400, Minibatch = 0.103299, Training Accuracy = 0.95312
Iterations :  44800, Minibatch = 0.144201, Training Accuracy = 0.94531
Iterations :  51200, Minibatch = 0.120480, Training Accuracy = 0.96875
Iterations :  57600, Minibatch = 0.107253, Training Accuracy = 0.96875
Iterations :  64000, Minibatch = 0.085129, Training Accuracy = 0.96875
Iterations :  70400, Minibatch = 0.062726, Training Accuracy = 0.99219
Iterations :  76800, Minibatch = 0.093519, Training Accuracy = 0.97656
Iterations :  83200, Minibatch = 0.031503, Training Accuracy = 0.98438
Iterations :  89600, Minibatch = 0.128479, Training Accuracy = 0.95312
Iterati